In [2]:
import numpy as np
import pandas as pd
import openpyxl
import yfinance as yf
from scipy.optimize import minimize

In [3]:
wb = openpyxl.load_workbook('PortfolioManagement.xlsx')

client_name = "VIJAY"

ws = wb[client_name]

# Assuming the first dataframe is from A1 to B16
stats_info = {}
for row in ws.iter_rows(min_row=1, max_row=16, min_col=1, max_col=2, values_only=True):
    stats_info[row[0]] = row[1]
stats_info

{'Client Name': 'ESHANI',
 'Start Date': datetime.datetime(2023, 4, 27, 0, 0),
 'Current Date': datetime.datetime(2024, 3, 31, 0, 0),
 'Benchmark': '^NSEI',
 'Risk-Free Rate': 0.0704,
 'Investment Amount': '=SUM(Table35[Invested Amount])',
 'Current Investment Value': '=SUM(H20:H83)',
 'Wallet Balance': 0,
 'Investment P/L': '=B7-B6',
 'Total P/L': '=SUM(Table35[Actual Return])',
 'Total Investment': '=B6+B8',
 'Net Return': '=B10/B11',
 'Portfolio Beta': '=SUMPRODUCT(Table35[Beta],Table35[Weights])',
 'Benchmark Return': 0.2462649792761106,
 "Jensen's Alpha": '=B12-(B5+B13*(B14-B5))',
 'Treynor Ratio': '=(B12-B5)/B13'}

In [4]:
df = pd.read_excel('PortfolioManagement.xlsx', sheet_name=client_name, skiprows=18)
df = df[df['Weights'] > 0]
df.head()

,Company name,Industry,Ticker,Beta,Category,Shares,Invested Amount,Current Value,Actual Return,% Absolute Rtn.,Expected Rtn,Weights,Optimal Weights,Current Price
0,ICICI Pru Nifty Bank ETF-G,ETF,BANKIETF,0.9157,NaN,9770,450976.22,464270.40,13294.18,0.029,0.231440,0.151965,0.136364,47.52
1,State Bank of India,Banking Services,SBIN,1.3941,LT,405,245473.01,304701.75,59228.74,0.241,0.315573,0.099735,0.099735,752.35
2,ICICI Bank,Banking Services,ICICIBANK,1.3002,LT,229,219997.85,250365.70,30975.85,0.141,0.299060,0.081950,0.081950,1093.30
3,JIO Financial Services,Investment Banking & Investment Services,JIOFIN,0.7040,AGG,557,160755.89,197038.75,38752.19,0.212,0.194209,0.064495,0.064495,353.75
4,Tata Motors,Automobiles & Auto Parts,TATAMOTORS,1.7889,LT,29,107421.46,194588.80,75204.49,0.630,0.385005,0.063693,0.063693,992.80


In [5]:
# Define the objective function to maximize Treynor ratio
def treynor_max(weights, returns, beta, risk_free_rate):
    portfolio_return = np.dot(returns, weights)
    portfolio_beta = np.dot(beta, weights)
    treynor_ratio = (portfolio_return - risk_free_rate) / portfolio_beta
    return -treynor_ratio  # Minimize the negative Treynor ratio

# Extract necessary data
returns = df['Actual Return']/df['Invested Amount']
risk_free_rate = stats_info.get("Risk-Free Rate", None)
beta = df["Beta"]
weights = df['Weights'].to_list()

In [6]:
# Define constraints
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

# Define bounds for each variable (weight)
bounds = [(0, 3/len(returns)) for _ in range(len(returns))]

# Perform optimization
result = minimize(treynor_max, weights, args=(returns, beta, risk_free_rate),
                method='SLSQP', bounds=bounds, constraints=constraints)

# Extract optimal weights
optimal_weights = result.x

# Assign optimal weights to the DataFrame
df['Optimized Weights'] = optimal_weights
df

c:\Users\siddh\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


,Company name,Industry,Ticker,Beta,Category,Shares,Invested Amount,Current Value,Actual Return,% Absolute Rtn.,Expected Rtn,Weights,Optimal Weights,Current Price,Optimized Weights
0,ICICI Pru Nifty Bank ETF-G,ETF,BANKIETF,0.9157,NaN,9770,450976.22,464270.40,13294.18,0.029,0.231440,0.151965,0.136364,47.52,0.136364
1,State Bank of India,Banking Services,SBIN,1.3941,LT,405,245473.01,304701.75,59228.74,0.241,0.315573,0.099735,0.099735,752.35,0.099735
2,ICICI Bank,Banking Services,ICICIBANK,1.3002,LT,229,219997.85,250365.70,30975.85,0.141,0.299060,0.081950,0.081950,1093.30,0.081950
3,JIO Financial Services,Investment Banking & Investment Services,JIOFIN,0.7040,AGG,557,160755.89,197038.75,38752.19,0.212,0.194209,0.064495,0.064495,353.75,0.064495
4,Tata Motors,Automobiles & Auto Parts,TATAMOTORS,1.7889,LT,29,107421.46,194588.80,75204.49,0.630,0.385005,0.063693,0.063693,992.80,0.063693
5,Sun Pharmaceutical Industries,Pharmaceuticals,SUNPHARMA,0.7197,LT,104,100450.32,168537.20,69386.88,0.691,0.196970,0.055166,0.055166,1620.55,0.055166
6,Bank of Baroda,Banking Services,BANKBARODA,1.3568,AGG,583,150404.77,153941.15,3536.36,0.024,0.309014,0.050388,0.050388,264.05,0.050388
7,Hindustan Unilever,Personal & Household Products & Services,HINDUNILVR,0.1950,LT,60,149083.08,135861.00,-12700.08,-0.085,0.104694,0.044470,0.044470,2264.35,0.044470
8,Asian Paints,Chemicals,ASIANPAINT,0.6343,LT,45,147136.53,128103.75,-18883.43,-0.128,0.181951,0.041931,0.041931,2846.75,0.041931
9,Indiabulls Real Estate,Real Estate Operations,IBREALEST,2.2488,NaN,1000,117555.05,115950.00,2265.08,0.013,0.465885,0.037953,0.037953,115.95,0.037953
